In [1]:
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as gnn
import numpy as np
import pandas as pd
from torch_geometric.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
from torch_geometric.utils import negative_sampling
from torch_geometric.nn import GCNConv 
from torch.optim.lr_scheduler import StepLR
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_auc_score, ndcg_score
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [5]:
class TwitterDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(TwitterDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['twitter_combined.txt', 'twitter_features.csv']

    @property
    def processed_file_names(self):
        return ['twitter_data.pt']

    def download(self):
        # In this case, you're not downloading any data,
        # but you could add code to download your data here if it's not local
        pass

    def process(self):
        # Load the graph data
        edges = pd.read_csv(self.raw_paths[0], delimiter=' ', header=None)
        features = pd.read_csv(self.raw_paths[1], delimiter=' ', header=None)

        # Create the graph
        edge_index = torch.tensor(edges.values, dtype=torch.long).t().contiguous()
        x = torch.tensor(features.values, dtype=torch.float)
    
        data = Data(x=x, edge_index=edge_index)

        data, slices = self.collate([data])
        torch.save((data, slices), self.processed_paths[0])

In [6]:
dataset = TwitterDataset(root='.')
data = dataset[0]

Processing...


ParserError: Error tokenizing data. C error: Expected 1 fields in line 2, saw 13
